
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session.                                                                                                 |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0).                               |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |  Changes the session type to Glue ETL.                                                                                                                    |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X.                                                                           |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer.                      |

In [19]:
%stop_session

Stopping session: ebdfb618-26ab-401e-803b-148309ec0237
Stopped session.


In [33]:
%connections hudi-connection
%glue_version 3.0
%region us-east-1
%worker_type G.1X
%number_of_workers 3
%spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer

You are already connected to a glueetl session ed10ba60-973b-4e30-ba22-d9dab9bb6428.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Connections to be included:
hudi-connection


You are already connected to a glueetl session ed10ba60-973b-4e30-ba22-d9dab9bb6428.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 3.0


You are already connected to a glueetl session ed10ba60-973b-4e30-ba22-d9dab9bb6428.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous region: us-east-1
Setting new region to: us-east-1
Reauthenticating Glue client with new region: us-east-1
IAM role has been set to arn:aws:iam::043916019468:role/Lab3. Reauthenticating.
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::043916019468:role/Lab3
Authentication done.
Region is set to: us-east-1


You are already connected to a glueetl session ed10ba60-973b-4e30-ba22-d9dab9bb6428.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session ed10ba60-973b-4e30-ba22-d9dab9bb6428.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 3
Setting new number of workers to: 3
Previous Spark configuration: spark.serializer=org.apache.spark.serializer.KryoSerializer
Setting new Spark configuration to: spark.serializer=org.apache.spark.serializer.KryoSerializer


In [17]:
try:
    import sys
    from awsglue.transforms import *
    from awsglue.utils import getResolvedOptions
    from pyspark.context import SparkContext
    from awsglue.context import GlueContext
    from awsglue.job import Job
    from pyspark.sql.session import SparkSession
    from awsglue.dynamicframe import DynamicFrame
    from pyspark.sql.functions import col, to_timestamp, monotonically_increasing_id, to_date, when
    from pyspark.sql.functions import *
    from awsglue.utils import getResolvedOptions
    from pyspark.sql.types import *
    from datetime import datetime
    import boto3
    from functools import reduce
except Exception as e:
    pass

In [18]:
spark = SparkSession.builder.config('spark.serializer','org.apache.spark.serializer.KryoSerializer').config('spark.sql.hive.convertMetastoreParquet','false').config('spark.sql.legacy.pathOptionBehavior.enabled', 'true').getOrCreate()
sc = spark.sparkContext
glueContext = GlueContext(sc)
job = Job(glueContext)
logger = glueContext.get_logger()

In [20]:
ReadDF = glueContext.create_dynamic_frame.from_catalog(
    database="qadb",
    table_name="table_name_oltp",
    transformation_ctx="ReadDF",
)

In [21]:
spark_df = ReadDF.toDF()

In [22]:
spark_df.show(2)

+--------------------+---------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+---------+------------+----+-----+---+
|                city|last_name|first_name|                  sk|                text|             address|                  pk|               state|awsRegion|             eventID|eventName| eventSource|year|month|day|
+--------------------+---------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+---------+------------+----+-----+---+
|15754 Dudley Land...|      Ray|      Eric|26be149e-d90a-422...|Republican enviro...|15754 Dudley Land...|26be149e-d90a-422...|Republican enviro...|us-east-1|d4cca531-6fea-453...|   REMOVE|aws:dynamodb|2022|   12| 19|
|5493 Isabel Hill
...|  Hampton|    Ashley|d8ccf3e0-9a6e-41d...|Attention serious...|5493 Isabel Hill
...|d8ccf3e0-9a6e-41d...|A

In [23]:
cols = ("eventName","awsRegion")
spark_df = spark_df.drop(*cols)

In [24]:
spark_df.show(1)

+--------------------+---------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+----+-----+---+
|                city|last_name|first_name|                  sk|                text|             address|                  pk|               state|             eventID| eventSource|year|month|day|
+--------------------+---------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+----+-----+---+
|15754 Dudley Land...|      Ray|      Eric|26be149e-d90a-422...|Republican enviro...|15754 Dudley Land...|26be149e-d90a-422...|Republican enviro...|d4cca531-6fea-453...|aws:dynamodb|2022|   12| 19|
+--------------------+---------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+----+-----+---+
only showi

In [26]:
db_name = "qadb"
table_name="hudi_table"
recordkey = 'pk'
precombine = 'sk'
path = "s3://jt-soumil-data-qa-dynamodb-1995/hudi/hudi_tables/"

In [27]:
pacheHudiConnector0101forAWSGlue30_node1671045598524 = (
    glueContext.write_dynamic_frame.from_options(
        frame=DynamicFrame.fromDF(spark_df, glueContext,"glue_df"),
        connection_type="marketplace.spark",
        connection_options={
            "path": path,
            "connectionName": "hudi-connection",

            "hoodie.datasource.write.storage.type": "COPY_ON_WRITE",
            'className': 'org.apache.hudi',
            'hoodie.table.name': table_name,
            'hoodie.datasource.write.recordkey.field': recordkey,
            'hoodie.datasource.write.table.name': table_name,
            'hoodie.datasource.write.operation': 'upsert',
            'hoodie.datasource.write.precombine.field': precombine,


            'hoodie.datasource.hive_sync.enable': 'true',
            "hoodie.datasource.hive_sync.mode":"hms",
            'hoodie.datasource.hive_sync.sync_as_datasource': 'false',
            'hoodie.datasource.hive_sync.database': db_name,
            'hoodie.datasource.hive_sync.table': table_name,
            'hoodie.datasource.hive_sync.use_jdbc': 'false',
            'hoodie.datasource.hive_sync.partition_extractor_class': 'org.apache.hudi.hive.MultiPartKeysValueExtractor',
            'hoodie.datasource.write.hive_style_partitioning': 'true',
        },
        transformation_ctx="glue_df",
    )
)